In [2]:
import numpy as np

from database.models import InstancesN8, InstancesN12
from tfim_sk_infd.models.Jij import Jij
from tfim_sk_infd.models.SKSpinGlass import SKSpinGlass
import data_service
from scipy.sparse import linalg as spla
from scipy import sparse

import time




In [14]:
# iterate through random seeds
# Look at 3 low h values 1e-3, 1e-4, 1e-5
# Look at sparse 1Q and sparse complete
# Get fidelity with previous
# Get fidelity with other method
# Get sum of prob amplitudes for both

N = 12
M = 2**N

# Spin inversion operator
U = np.zeros((M, M))
for i in range(M):
    U[i][i] = 1 if i<(M)//2 else -1
    U[i][-i-1] = 1
U /= np.sqrt(2)
U = sparse.coo_matrix(U)



Instance = data_service.get_instance_class(N)

seeds = [26]
steps = 3

with data_service.get_session() as session:

    instances: list[InstancesN8 | InstancesN12] = (
        session.query(Instance).where(Instance.seed.in_(seeds)).all()
    )

    out_obj = {}

    for instance in instances:



        deg = len(instance.ground_states)

        sparse_fidelities_1q = np.zeros(steps)
        sparse_classical_gs_probs_1q = np.zeros(steps)
        
        sparse_fidelities_comp = np.zeros(steps)
        sparse_classical_gs_probs_comp = np.zeros(steps)

        hx = np.zeros(steps)
        comp_vs_1q_fids = np.zeros(steps)



        # Initialize spin glass
        Jij_obj = Jij(np.array(instance.Jij_matrix))
        SG = SKSpinGlass(Jij_obj)

        h = 1e-1
        sparse_psi0_1q = None
        sparse_psi0_comp = None

        for i in range(steps):

            # Calculate hamiltonian
            h /= 10
            H = SG.hamiltonian_at_h(h)

            

            ## SPARSE 1Q

            # UHU = np.dot(U,np.dot(H, U.T))
            UHU = U@H@U.T
            UHU1q = UHU[0:M//2, 0:M//2]
            start = time.time()
            E1q, v1q = spla.eigsh(UHU1q, k=1, which="SA", v0=sparse_psi0_1q)
            print("sparse 1q time", time.time()-start)
            sort_order = np.argsort(E1q)
            E1q = E1q[sort_order]
            v1q = v1q[:, sort_order]
            old_sparse_psi0_1q = sparse_psi0_1q
            sparse_psi0_1q = v1q[:, 0]
            sparse_psi0_1q_complete = U@(np.array(list(sparse_psi0_1q) + [0 for _ in range(M//2)]))
            

            sparse_fidelities_1q[i] = np.abs(np.vdot(sparse_psi0_1q, old_sparse_psi0_1q)) if old_sparse_psi0_1q is not None else 1
            sparse_classical_gs_probs_1q[i] = np.sum(sparse_psi0_1q_complete[instance.ground_states]**2)


            # UHU4q = UHU[M//2:, M//2:]
            # start = time.time()
            # E4q, v4q = spla.eigsh(UHU4q, k=1, which="SA", v0=sparse_psi0_4q)
            # print("sparse 4q time", time.time()-start)
            # sort_order = np.argsort(E4q)
            # E4q = E4q[sort_order]
            # v4q = v4q[:, sort_order]
            # old_sparse_psi0_4q = sparse_psi0_4q
            # sparse_psi0_4q = v4q[:, 0]
            # sparse_psi0_4q_complete = U@(np.array(list(sparse_psi0_4q) + [0 for _ in range(M//2)]))
            

            # sparse_fidelities_4q[i] = np.abs(np.vdot(sparse_psi0_4q, old_sparse_psi0_4q)) if old_sparse_psi0_4q is not None else 1
            # sparse_classical_gs_probs_4q[i] = np.sum(sparse_psi0_4q_complete[instance.ground_states]**2)


            ## SPARSE COMPLETE
            start = time.time()
            Ecomp, vcomp = spla.eigsh(H, k=1, which="SA", v0=sparse_psi0_comp)
            print("sparse complete time", time.time()-start)
            sort_order = np.argsort(Ecomp)
            Ecomp = Ecomp[sort_order]
            vcomp = vcomp[:, sort_order]
            old_sparse_psi0_comp = sparse_psi0_comp
            sparse_psi0_comp = vcomp[:, 0]
            

            sparse_fidelities_comp[i] = np.abs(np.vdot(sparse_psi0_comp, old_sparse_psi0_comp)) if old_sparse_psi0_comp is not None else 1
            sparse_classical_gs_probs_comp[i] = np.sum(sparse_psi0_comp[instance.ground_states]**2)


            
            hx[i] = h
            comp_vs_1q_fids[i] = np.abs(np.vdot(sparse_psi0_1q_complete, sparse_psi0_comp))
            
        out_obj[instance.seed] = {
            "hx": hx,
            "sparse_fidelities_1q": sparse_fidelities_1q, 
            "sparse_classical_gs_probs_1q": sparse_classical_gs_probs_1q, 
            "sparse_fidelities_comp": sparse_fidelities_comp, 
            "sparse_classical_gs_probs_comp": sparse_classical_gs_probs_comp, 
            "comp_vs_1q_fids": comp_vs_1q_fids, 
        }
                

    session.close()

sparse 1q time 0.007184028625488281
sparse complete time 0.029005050659179688
sparse 1q time 0.005700111389160156
sparse complete time 0.010892868041992188
sparse 1q time 0.003459930419921875
sparse complete time 0.004481077194213867


In [6]:

sparse_psi0_1q_complete


matrix([[1.30508683e-11, 3.18164092e-14, 9.87520823e-14, ...,
         9.87520823e-14, 3.18164092e-14, 1.30508683e-11]])

In [18]:
(deg//2) + 1

4

In [39]:
low_fidelity_comp = 0
low_fidelity_1q = 0
for i in range(100):
    low_fidelity_comp += 1 if sum(out_obj[i]["sparse_fidelities_comp"]) < 2.9 else 0
    low_fidelity_1q += 1 if sum(out_obj[i]["sparse_fidelities_1q"]) < 2.9 else 0


In [40]:
print(low_fidelity_comp)
print(low_fidelity_1q)

37
0


In [19]:
N = 2
M = 2**N

U = np.zeros((M, M))
for i in range(M):
    U[i][i] = 1 if i<(M)//2 else -1
    U[i][-i-1] = 1
U /= np.sqrt(2)

for row in U:
    print(row)

[0.70710678 0.         0.         0.70710678]
[0.         0.70710678 0.70710678 0.        ]
[ 0.          0.70710678 -0.70710678  0.        ]
[ 0.70710678  0.          0.         -0.70710678]


In [20]:
# Eigen energy gap E1 - E0
# Fidelity between h values
# total classical gs prob

In [22]:
start = time.time()
U = np.zeros((M, M))
for i in range(M):
    U[i][i] = 1 if i<(M)//2 else -1
    U[i][-i-1] = 1
U /= np.sqrt(2)
U = sparse.coo_matrix(U)
print(time.time() - start)

0.001332998275756836


In [23]:
start = time.time()
U = sparse.coo_matrix((M,M))
for i in range(M):
    U[i][i] = 1 if i<(M)//2 else -1
    U[i][-i-1] = 1
U /= np.sqrt(2)
print(time.time() - start)

TypeError: 'coo_matrix' object is not subscriptable